<a href="https://colab.research.google.com/github/PARTHIBAN-007/Hands-ON-LLMs/blob/main/11_FineTuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Data

In [2]:
from datasets import load_dataset

tomatoes = load_dataset("rotten_tomatoes")
train_data , test_data = tomatoes["train"] ,tomatoes["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

# Supervised Classification

In [3]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
model_id = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels= 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer =tokenizer)

def text_preprocess(examples):
  return tokenizer(examples["text"],truncation=True)

tokenized_train = train_data.map(text_preprocess,batched=True)
tokenized_test = test_data.map(text_preprocess,batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  logits , labels = eval_pred
  predictions = np.argmax(logits,axis=-1)

  load_f1 = evaluate.load("f1")
  f1 = load_f1.compute(predictions=predictions,references=labels)["f1"]
  return {"f1":f1}

In [6]:
from transformers import TrainingArguments , Trainer

training_arguments = TrainingArguments(
    "model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs = 1,
    weight_decay = 0.01,
    save_strategy="epoch",
)


trainer = Trainer(
    model = model,
    args = training_arguments ,
    train_dataset = tokenized_train ,
    eval_dataset  = tokenized_test ,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

<ipython-input-6-05e234994351>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.395100


TrainOutput(global_step=534, training_loss=0.3909175940667199, metrics={'train_runtime': 126.4989, 'train_samples_per_second': 67.431, 'train_steps_per_second': 4.221, 'total_flos': 213940121334480.0, 'train_loss': 0.3909175940667199, 'epoch': 1.0})

In [8]:
trainer.evaluate()

{'eval_loss': 0.36170995235443115,
 'eval_f1': 0.8555555555555555,
 'eval_runtime': 4.5974,
 'eval_samples_per_second': 231.871,
 'eval_steps_per_second': 14.574,
 'epoch': 1.0}

## Freeze Layers

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels= 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
for name ,param in model.named_parameters():
  print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [11]:
for name ,param in  model.named_parameters():
  if name.startswith("classifier"):
    param.requires_grad =True
  else:
    param.requires_grad = False

In [12]:
for name , param in model.named_parameters():
  print(f"Parameter : {name} -------- {param.requires_grad}")

Parameter : bert.embeddings.word_embeddings.weight -------- False
Parameter : bert.embeddings.position_embeddings.weight -------- False
Parameter : bert.embeddings.token_type_embeddings.weight -------- False
Parameter : bert.embeddings.LayerNorm.weight -------- False
Parameter : bert.embeddings.LayerNorm.bias -------- False
Parameter : bert.encoder.layer.0.attention.self.query.weight -------- False
Parameter : bert.encoder.layer.0.attention.self.query.bias -------- False
Parameter : bert.encoder.layer.0.attention.self.key.weight -------- False
Parameter : bert.encoder.layer.0.attention.self.key.bias -------- False
Parameter : bert.encoder.layer.0.attention.self.value.weight -------- False
Parameter : bert.encoder.layer.0.attention.self.value.bias -------- False
Parameter : bert.encoder.layer.0.attention.output.dense.weight -------- False
Parameter : bert.encoder.layer.0.attention.output.dense.bias -------- False
Parameter : bert.encoder.layer.0.attention.output.LayerNorm.weight -------

In [13]:
from transformers import TrainingArguments , Trainer

trainer = Trainer(
    model=model,
    args = training_arguments ,
    train_dataset = tokenized_train ,
    eval_dataset = tokenized_test ,
    tokenizer = tokenizer ,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

<ipython-input-13-fb1e9d2c1687>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.695700


TrainOutput(global_step=534, training_loss=0.6950046096401714, metrics={'train_runtime': 36.2601, 'train_samples_per_second': 235.245, 'train_steps_per_second': 14.727, 'total_flos': 213940121334480.0, 'train_loss': 0.6950046096401714, 'epoch': 1.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.6907070875167847,
 'eval_f1': 0.6395262768319763,
 'eval_runtime': 4.4175,
 'eval_samples_per_second': 241.312,
 'eval_steps_per_second': 15.167,
 'epoch': 1.0}

## Freeze blocks 1-5

In [15]:
for index , (name,parameters) in enumerate(model.named_parameters()):
  print(f"Parmeter : {index}{name} ----- {parameters.requires_grad}")

Parmeter : 0bert.embeddings.word_embeddings.weight ----- False
Parmeter : 1bert.embeddings.position_embeddings.weight ----- False
Parmeter : 2bert.embeddings.token_type_embeddings.weight ----- False
Parmeter : 3bert.embeddings.LayerNorm.weight ----- False
Parmeter : 4bert.embeddings.LayerNorm.bias ----- False
Parmeter : 5bert.encoder.layer.0.attention.self.query.weight ----- False
Parmeter : 6bert.encoder.layer.0.attention.self.query.bias ----- False
Parmeter : 7bert.encoder.layer.0.attention.self.key.weight ----- False
Parmeter : 8bert.encoder.layer.0.attention.self.key.bias ----- False
Parmeter : 9bert.encoder.layer.0.attention.self.value.weight ----- False
Parmeter : 10bert.encoder.layer.0.attention.self.value.bias ----- False
Parmeter : 11bert.encoder.layer.0.attention.output.dense.weight ----- False
Parmeter : 12bert.encoder.layer.0.attention.output.dense.bias ----- False
Parmeter : 13bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parmeter : 14bert.encoder.laye

In [16]:
model_id = "bert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)


for index , (name ,parameters) in enumerate(model.named_parameters()):
  if index<165:
    param.requires_grad = False


trainer = Trainer(
    model = model ,
    args = training_arguments ,
    train_dataset = tokenized_train ,
    eval_dataset = tokenized_test ,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-375c3ac37705>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.385000


{'eval_loss': 0.36175286769866943,
 'eval_f1': 0.8576675849403123,
 'eval_runtime': 4.2464,
 'eval_samples_per_second': 251.035,
 'eval_steps_per_second': 15.778,
 'epoch': 1.0}

## Few Shot Classification

In [17]:
from setfit import sample_dataset

sampled_train_data = sample_dataset(tomatoes['train'],num_samples= 16)

In [18]:
from setfit  import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [19]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

args = SetFitTrainingArguments(
    num_epochs = 3,
    num_iterations = 20
)

args.eval_strategy = args.evaluation_strategy

trainer = SetFitTrainer(
    model = model ,
    args = args ,
    train_dataset = sampled_train_data ,
    eval_dataset = test_data ,
    metric ="f1"
)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [20]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running training *****
  Num unique pairs = 1280
  Batch size = 16
  Num epochs = 3


Step,Training Loss,Validation Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'f1': 0.845703125}

In [21]:
model.model_head

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LogisticRegression()

## MLM

In [22]:
from transformers import AutoTokenizer , AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [23]:
def preprocess_function(examples):
  return tokenizer(examples['text'],truncation=True)


tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns("label")
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test = tokenized_test.remove_columns("label")

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [24]:
from transformers import DataCollatorForLanguageModeling

data_collator  = DataCollatorForLanguageModeling(
    tokenizer =tokenizer ,
    mlm = True,
    mlm_probability = 0.15
)

In [25]:
training_args = TrainingArguments(
    "model",
     learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-25-d11664da27eb>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [26]:
tokenizer.save_pretrained("mlm")
trainer.train()
trainer.evaluate()
model.save_pretrained("mlm")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Step,Training Loss
500,2.517300
1000,2.405500
1500,2.300300
2000,2.172400
2500,2.111300
3000,2.106900
3500,2.043900
4000,2.045500
4500,2.010700
5000,2.026900


In [27]:
from transformers import pipeline

mask_filler = pipeline("fill-mask",model="bert-base-uncased")
preds = mask_filler("What a Horrible  [MASK]! ")
for pred in preds:
  print(f">>> {pred['sequence']}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fro

>>> what a horrible idea!
>>> what a horrible thing!
>>> what a horrible day!
>>> what a horrible story!
>>> what a horrible dream!


In [28]:
mask_filler  = pipeline("fill-mask",model="mlm")
preds = mask_filler("What a horrible [MASK]!")
for pred in preds:
  print(f">>> {pred['sequence']}")

Device set to use cuda:0


>>> what a horrible movie!
>>> what a horrible film!
>>> what a horrible thing!
>>> what a horrible idea!
>>> what a horrible picture!


## NAMED ENTITY RECOGNITION

In [29]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import numpy as np

In [30]:
dataset = load_dataset("conll2003", trust_remote_code=True)

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [31]:
example = dataset["train"][243]
example

{'id': '243',
 'tokens': ['Gateway', 'Data', 'Sciences', 'Q2', 'net', 'rises', '.'],
 'pos_tags': [22, 23, 23, 22, 16, 24, 7],
 'chunk_tags': [11, 12, 12, 12, 12, 12, 0],
 'ner_tags': [3, 4, 4, 0, 0, 0, 0]}

In [32]:
label2id = {
    'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4,
    'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8
}
id2label = {index: label for label, index in label2id.items()}
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [33]:
from transformers import AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
token_ids = tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
sub_tokens = tokenizer.convert_ids_to_tokens(token_ids)
sub_tokens

['[CLS]',
 'Gateway',
 'Data',
 'Sciences',
 'Q',
 '##2',
 'net',
 'rises',
 '.',
 '[SEP]']

In [35]:
def align_labels(examples):
    token_ids = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = examples["ner_tags"]

    updated_labels = []
    for index, label in enumerate(labels):

        word_ids = token_ids.word_ids(batch_index=index)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:

            if word_idx != previous_word_idx:

                previous_word_idx = word_idx
                updated_label = -100 if word_idx is None else label[word_idx]
                label_ids.append(updated_label)

            elif word_idx is None:
                label_ids.append(-100)

            else:
                updated_label = label[word_idx]
                if updated_label % 2 == 1:
                    updated_label += 1
                label_ids.append(updated_label)

        updated_labels.append(label_ids)

    token_ids["labels"] = updated_labels
    return token_ids

tokenized = dataset.map(align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [36]:
print(f"Original: {example['ner_tags']}")
print(f"Updated: {tokenized['train'][243]['labels']}")

Original: [3, 4, 4, 0, 0, 0, 0]
Updated: [-100, 3, 4, 4, 0, 0, 0, 0, 0, -100]


In [37]:
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)

    true_predictions = []
    true_labels = []
    for prediction, label in zip(predictions, labels):

      for token_prediction, token_label in zip(prediction, label):
        if token_label != -100:
          true_predictions.append([id2label[token_prediction]])
          true_labels.append([id2label[token_label]])

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {"f1": results["overall_f1"]}

In [38]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

trainer.evaluate()

<ipython-input-38-94a41779e513>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.230600


{'eval_loss': 0.1380533128976822,
 'eval_f1': 0.9029931712298168,
 'eval_runtime': 12.708,
 'eval_samples_per_second': 271.718,
 'eval_steps_per_second': 16.997,
 'epoch': 1.0}

In [39]:
from transformers import pipeline
trainer.save_model("ner_model")
token_classifier = pipeline(
    "token-classification",
    model="ner_model",
)
token_classifier("My name is Maarten.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Device set to use cuda:0


[{'entity': 'B-PER',
  'score': 0.98172003,
  'index': 4,
  'word': 'Ma',
  'start': 11,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.9258569,
  'index': 5,
  'word': '##arte',
  'start': 13,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.9556565,
  'index': 6,
  'word': '##n',
  'start': 17,
  'end': 18}]